# Pytorch - Redes Neuronales

In [ ]:
import torch

In [ ]:
D_in, H, D_out = 784, 55, 10

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

In [ ]:
outputs = model(torch.randn(64, 784))
outputs.shape

torch.Size([64, 10])

In [ ]:
model.to("cuda")

Sequential(
  (0): Linear(in_features=784, out_features=55, bias=True)
  (1): ReLU()
  (2): Linear(in_features=55, out_features=10, bias=True)
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = np.loadtxt("/content/drive/MyDrive/Colab Notebooks/DATASETS/fashion-mnist_train.csv", delimiter=',',skiprows=1)
# print(data)
X, Y = data[:, 1:], data[:, 0]
print(X.shape)
print(Y.shape)

(60000, 784)
(60000,)


In [ ]:
import numpy as np

# normalización y split

X_train, X_test, y_train, y_test = X[:50000] / 255., X[50000:] / 255., Y[:50000].astype(np.int), Y[50000:].astype(np.int)

<ipython-input-131-a86473860af5>:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_train, X_test, y_train, y_test = X[:50000] / 255., X[50000:] / 255., Y[:50000].astype(np.int), Y[50000:].astype(np.int)


In [ ]:
# función de pérdida y derivada

def softmax(x):
    return torch.exp(x) / torch.exp(x).sum(axis=-1,keepdims=True)

def cross_entropy(output, target):
    logits = output[torch.arange(len(output)), target]
    loss = - logits + torch.log(torch.sum(torch.exp(output), axis=-1))
    loss = loss.mean()
    return loss

In [ ]:
print(X)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
model.cuda()

Sequential(
  (0): Linear(in_features=784, out_features=55, bias=True)
  (1): ReLU()
  (2): Linear(in_features=55, out_features=10, bias=True)
)

In [ ]:
# convertimos datos a tensores y copiamos en gpu

X_t = torch.from_numpy(X_train).float().cuda()
Y_t = torch.from_numpy(y_train).long().cuda()

# bucle entrenamiento
epochs = 100
lr = 0.6
log_each = 10
l = []
for e in range(1, epochs + 1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = cross_entropy(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    model.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    with torch.no_grad():
        for param in model.parameters():
            param -= lr * param.grad

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

Epoch 10/100 Loss 2.04888
Epoch 20/100 Loss 1.87088
Epoch 30/100 Loss 1.68361
Epoch 40/100 Loss 1.57030
Epoch 50/100 Loss 1.58525
Epoch 60/100 Loss 1.58935
Epoch 70/100 Loss 1.55947
Epoch 80/100 Loss 1.50843
Epoch 90/100 Loss 1.44728
Epoch 100/100 Loss 1.39517


In [ ]:
from sklearn.metrics import accuracy_score

def evaluate(x):
    model.eval()
    y_pred = model(x)
    y_probas = softmax(y_pred)
    return torch.argmax(y_probas, axis=1)

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

0.6838